Modify the scratch code of K-means clustering in our lecture:
- Modify so it print out the total within-cluster variation.  Then try to run several k and identify which k is best.
- Since k-means can be slow due to its pairwise computations, let's implement a mini-batch k-means in which the cluster is create using only partial subset of samples.
- Put everything into a class

Mini-Batch will rarely converge, thus it is important to add a max_iteration or some tolerance.  Last, theoretically speaking, Mini-Batch will never perform better in terms of accuracy when compare to K-means, but it is very close to optimal but will almost always beat K-means in terms of time given large dataset and a modest tolerance parameter.

In [2]:
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import make_blobs
from time import time
import numpy as np

X, y_true = make_blobs(n_samples=1500, centers=4,
                       cluster_std=0.60, random_state=0)
class Mini_KMeans:
    def __init__(self, k, replacement=True, batch_size=100, max_iter=100):
        self.k = k
        self.replacement=replacement
        self.batch_size = batch_size
        self.max_iter = max_iter
    
    def fit(self, X):
        m, n = X.shape

        #1. randomly choose k clusters from X
        rng = np.random.RandomState(99)
        i = rng.permutation(m)[:self.k]
        self.centers = X[i]

        #having max iter makes sure it will stop eventually
        for ix in np.arange(self.max_iter):
            random = rng.randint(m)
            X_batch = X[random:random+self.batch_size]

            #2. assign labels based on closest center
            labels = pairwise_distances_argmin(X_batch, self.centers)

            #3. find new centers
            new_centers = []
            for i in range(self.k):
                new_centers.append(X_batch[labels == i].mean(axis=0))

            #convert list to np.array; you can actually combine #3
            #with np.array in one sentence 
            new_centers = np.array(new_centers)

            #4 stopping criteria - if centers do not 
            #change anymore, we stop!
            #make sure to add rtol or atol since mini-batch does not converge
            if(np.allclose(self.centers, new_centers, rtol=0.2)):
                break
            else:
                self.centers = new_centers

        print(f"Done in {ix} iterations")

        #compute total within-variation score
        total_with_variation_score = 0
        labels = pairwise_distances_argmin(X, self.centers) #<---Note I use X here.  Why?
        for i in range(self.k):
            cluster_mean = X[labels==i].mean(axis=0)
            total_with_variation_score += ((X[labels==i] - cluster_mean)** 2).sum()
            
        print("Total with variation score: ", total_with_variation_score)

    def predict(self, X):
        return pairwise_distances_argmin(X, self.centers)

#main code

for k in range(2, 7):
    print(f"=====k = {k}")
    start = time()
    model = Mini_KMeans(k)
    model.fit(X)
    preds = model.predict(X)
    print(f"Fit and predict time {time() - start}")



=====k = 2
Done in 6 iterations
Total with variation score:  5859.027548790498
Fit and predict time 0.007968902587890625
=====k = 3
Done in 3 iterations
Total with variation score:  2849.7266714358066
Fit and predict time 0.00592803955078125
=====k = 4
Done in 9 iterations
Total with variation score:  1007.7374341654453
Fit and predict time 0.008007287979125977
=====k = 5
Done in 9 iterations
Total with variation score:  920.8127407429872
Fit and predict time 0.010268211364746094
=====k = 6
Done in 3 iterations
Total with variation score:  841.4034939553978
Fit and predict time 0.00701904296875
